#  Expected number of cases prior to detection by symptomatic testing




This notebook contains an example calculation of the number of cases one would expect before detection by symptom-led testing in an simple doubling process epidemic.  

User slider inputs:
- **doubling_time** is the doubling time (in days) of the number of *new* daily cases.  This is the only specification of the epidemic growth process.  There is no latent period or recovery, nor stochasticity or super-spreading, nor any other disease model.
- **post_infection_delay** is the number of days after intially being infected that an infected person who will have symptoms finds out that they are infectious - the day the outbreak is detected.  This might include an asymptomatic period of several days as well as a testing delay.  So, for example, if you believe that a person is infectious for two days before being symptomatic, and then takes 1 day to seek a test once symptoms appear, then you might set this at three days.
- **fraction_non_testing** is the fraction of infected people who will never develop symptoms, or will not seek a test for some other reason.  E.g. if you believe that half of your population would develop symptoms if infected and all of them would seek a test if they had symptoms, then this should be 0.5.  There is significant uncertainty about this number for COVID-19, and it is likely that this will vary by age class and social setting.  


The output plot gives the estimated probability (essentially via an implementation of a geometric distribution) of number-of-cases on the day of detection, given the assumptions set by the sliders.  We report the expectation and plot it as a red line.


*The code in this notebook is non-beautiful, and may be changed at any time.  It is intentionally simple and not especially Pythonic.*

In [10]:
import math
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import interact

def gen_prob_first_sympt_infected_on_day(cases, prob_asympt):
  dist = []
  prev_prob = [0]
  for i in range(len(cases)):
    undetect_today = (prob_asympt)**cases[i]
    detect_today= 1-undetect_today
    detect_first_today = (1-prev_prob[i])*detect_today
    dist.append(detect_first_today)
    prev_prob.append(prev_prob[i] + detect_first_today)
  return dist

def threshold_out_tail(cases, probs, thresh):
  for i in range(len(probs)):
    if probs[i] < thresh:
      return cases[:i], probs[:i]
  return cases,probs

def dist_num_cases_on_detection(cases, first_day_sympt_dist, delay):
  num_cases = []
  prob = []
  for i in range(len(cases)-delay):
    prob.append(first_day_sympt_dist[i])
    num_cases.append(sum(cases[:i+delay]))
  return num_cases,prob


def gen_number_cases_list(doubling_time, horizon):
  lambda_rate = 0.693/doubling_time
  cases = []
  for i in range(horizon):
    cases.append(math.exp(i*lambda_rate))
  return cases

def gen_prob_detect(cases, prob_asympt):
  prob_det = []
  for i in range(len(cases)):
    prob_det.append(1-(prob_asympt)**cases[i])
  return prob_det

def cumulative(list_name):
  cum_list = []
  for i in range(len(list_name)):
    cum_list.append(sum(list_name[:i+1]))
  return cum_list

tail_thresh = 0.00001
max_double=20
max_delay=7


def plot_the_graph(doubling_time, post_infection_delay, fraction_non_testing):
   cases = gen_number_cases_list(doubling_time, horizon=20)
   first_day_dist = gen_prob_first_sympt_infected_on_day(cases, fraction_non_testing)

   expect_cases,prob = dist_num_cases_on_detection(cases, first_day_dist, post_infection_delay)
   expect_cases,prob = threshold_out_tail(expect_cases,prob,tail_thresh)
#    print(expect_cases)
    
   cum_expect_cases = cumulative(expect_cases) 
#    print(cum_expect_cases)
   expectation = sum([a*b for a,b in zip(cum_expect_cases,prob)])
   
   print('Expected number of cases on day of detection ' + str(round(expectation, 2)))

   plt.plot(expect_cases, prob)
   plt.ylim(0, 0.75)
  #  plt.xlim(1,100)
  #  print(expect_cases)
  #  print(prob)
  #  print(sum(prob))
   plt.xlabel('Number of cases on day of detection')
   plt.ylabel('Probability')
   plt.axvline(x=expectation, color='red')
   plt.show()


interact(plot_the_graph, doubling_time=(1,max_double), post_infection_delay=(0,max_delay), fraction_non_testing=(0.05, 0.95));

interactive(children=(IntSlider(value=10, description='doubling_time', max=20, min=1), IntSlider(value=3, desc…